<center><img alt="" src="images/Cover.jpg"/></center> 

## <center><font color="blue">Text Preprocessing and Representations</font></center>

<h2 id="(C)-Taufik-Sutanto---2019" style="text-align: center;">(C) Taufik Sutanto - 2019</h2>
<h2 id="tau-data-Indonesia-~-https://tau-data.id" style="text-align: center;">tau-data Indonesia ~ <a href="https://tau-data.id" target="_blank"><span style="color: #0009ff;">https://tau-data.id</span></a></h2>

# Notes and Disclaimer

* This notebook is part of the free (open knowledge) eLearning course at: https://tau-data.id/courses/
* Some images are taken from several resources, we respect those images ownerships and put a reference/citation from where it is originated. Nevertheless, sometimes we are having trouble to find the origin of the image(s). If you are the owner of the image and would like the image taken-out (or want the citation to be revised) from this open knowledge course resources please contact us here with the details: https://tau-data.id/contact/  
* Unless stated otherwise, in general tau-data permit its resources to be copied and-or modified for non-commercial purposes. With condition proper acknowledgement/citation is given.

## Outline:

* Review
* Pendahuluan Natural Language Processing (NLP) dan Text Mining
* PreProcessing
* NLP 
* Vector Space Model

# Review
<p><img alt="" src="images/5_Clus-Clas.png" style="height:500px; width:766px" /></p>

<p><img alt="" src="images/5_clustering_benchmarks.png" style="height:500px; width:962px" /></p>

In [ ]:
# Installing Modules for Google Colab
try:
    !wget https://raw.githubusercontent.com/taufikedys/tau-data/master/taudata.py
    !wget https://raw.githubusercontent.com/taufikedys/tau-data/master/data/stopwords_id.txt
    !wget https://raw.githubusercontent.com/taufikedys/tau-data/master/data/stopwords_en.txt
    !wget https://github.com/taufikedys/tau-data/blob/master/data/all_indo_man_tag_corpus_model.crf.tagger
except:
    pass
!pip install unidecode
!pip install pyLDAvis
!pip install textblob
!pip install sastrawi
!pip install spacy
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm
import nltk
nltk.download('popular')

In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import taudata as tau
import time, numpy as np, matplotlib.pyplot as plt, networkx as nx, pandas as pd, seaborn as sns
import pyLDAvis, pyLDAvis.sklearn
from matplotlib.colors import ListedColormap
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from spacy.lang.id import Indonesian; nlp_id = Indonesian()  # Language Model
from spacy.lang.en import English; nlp_en = English()
from sklearn.datasets import fetch_20newsgroups
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from unidecode import unidecode
from nltk.tag import CRFTagger
from textblob import Word
from html import unescape
from nltk.tokenize import TweetTokenizer
import itertools, re, pickle

stemmer = StemmerFactory().create_stemmer()
pyLDAvis.enable_notebook()
sns.set(style="ticks", color_codes=True)
random_state = 170

# Natural Language Processing (NLP) - Pemrosesan Bahasa Alami (PBA):

<p>
&quot;<big><em>Sebuah cabang ilmu&nbsp;(AI/Computational Linguistik) yang mempelajari bagaimana&nbsp;bahasa (alami) manusia (terucap/tertulis) dapat dipahami dengan baik oleh komputer dan komputer dapat merespon dengan cara yang serupa ke manusia</em></big>&quot;.</p>

<p><img alt="" src="images/1_jarvis.jpg" style="height: 450px; width: 600px;" /></p>

<p><a href="https://www.turn-on.de/lifestyle/topliste/zehn-film-gadgets-die-wir-uns-im-wahren-leben-wuenschen-4413" target="_blank"><strong>[Image Source]</strong></a></p>


<p><strong>Aplikasi Umum NLP:</strong></p>

<ol>
	<li>Machine Translation (Misal&nbsp;https://translate.google.com/ )</li>
	<li>Information Retrieval (IR)&nbsp;(misal www.google.com, bing, elasticsearch, etc.)</li>
	<li>Man-Machine Interface (misal Siri, cortana, atau Alexa)</li>
</ol>


<p><strong>Apakah Perbedaan antara NLP dan Text Mining (TM)?</strong></p>

<p>TM (terkadang disebut Text Analytics) adalah sebuah pemrosesan teks (biasanya dalam skala besar) untuk menghasilkan (generate) informasi atau insights. Untuk menghasilkan informasi TM menggunakan beberapa metode, termasuk NLP. TM mengolah teks secara eksplisit, sementara NLP mencoba mencari makna latent (tersembunyi) lewat aturan bahasa (e.g. grammar/idioms/Semantics).<br />
<strong>Contoh aplikasi TM</strong> : Clustering, Klasifikasi, Social Media Analytics (SMA).</p>


<p><img alt="https://www.kdnuggets.com/2017/11/framework-approaching-textual-data-tasks.html" src="images/1_NLP_TextMining.jpg" style="height: 470px; width: 600px;" /></p>

<p>[image source: <a href="https://www.elsevier.com/books/practical-text-mining-and-statistical-analysis-for-non-structured-text-data-applications/miner/978-0-12-386979-1" target="_blank">Gary M.:&quot;Practical Text Mining and Statistical Analysis for Non-structured Text Data Applications&quot;</a>]</p>


<p><img alt="" src="images/1_Text_Analytics.jpg" style="height: 451px; width: 600px;" /></p>

<p>[Image Source: <a href="http://www.pearson.com.au/products/S-Z-Turban-Sharda/Business-Intelligence-and-Analytics-Systems-for-Decision-Support-Global-Edition/9781292009209?R=9781292009209" target="_blank">Efraim T. &quot;Business Intelligence and Analytics: Systems for Decision Support, Global Edition (10e)</a>&quot;]</p>


# PreProcessing I

* Tokenisasi
* Stemming dan Lemma
* Pos tag 

<p><strong>Tokenisasi</strong></p>

<p>Tokenisasi adalah pemisahan kata, simbol, frase, dan entitas penting lainnya (yang disebut sebagai token) dari sebuah teks untuk kemudian di analisa lebih lanjut. Token dalam NLP sering dimaknai dengan &quot;sebuah kata&quot;, walau tokenisasi juga bisa dilakukan ke kalimat, paragraf, atau entitas penting lainnya (misal suatu pola string DNA di Bioinformatika).</p>

<p><strong>Mengapa perlu tokenisasi?</strong></p>

<ul>
	<li>Langkah penting dalam preprocessing, menghindari kompleksitas mengolah langsung pada string asal.</li>
	<li>Menghindari masalah (semantic) saat pemrosesan model-model natural language.</li>
	<li>Suatu tahapan sistematis dalam merubah unstructured (text) data ke bentuk terstruktur yang lebih mudah di olah.</li>
</ul>

<p><img alt="" src="images\2_Pipeline_Tokenization.png" style="height:300px; width:768px" /><br />
[<a href="https://www.softwareadvice.com/resources/what-is-text-analytics/" target="_blank"><strong>Image Source</strong></a>]</p>


## Tokenisasi dengan <font color="blue"> TextBlob</font>
<strong>Kelebihan</strong>:</p>
<ol>
	<li>Sederhana &amp; mudah untuk digunakan/pelajari.</li>
	<li>Textblob objects punya behaviour/properties yang sama dengan string di Python.</li>
	<li>TextBlob dibangun dari kombinasi modul NLTK dan (Clips) Pattern yang reliable</li>
</ol>

<p><strong>Kekurangan</strong>:</p>
<ol>
	<li>Tidak secepat Modul TextMining/NLP lain</li>
	<li>Language Model terbatas (tidak mendukung bahasa Indonesia): English, German, French</li>
</ol>

<p>*Blob : Binary large Object</p>

In [ ]:
# Tokenizing di TextBlob
T = "Hello, Mr. Man. He smiled!! This, i.e. that, is it."
kalimatS = TextBlob(T).sentences
print([str(kalimat) for kalimat in kalimatS])
print(TextBlob(T).words)

In [ ]:
T.split()

## Tokenisasi tidak hanya language dependent, tapi juga environment dependent

<p>Tokenization sebenarnya tidak sesederhana memisahkan berdasarkan spasi dan removing symbol. Sebagai contoh dalam bahasa Jepang/Cina/Arab suatu kata bisa terdiri dari beberapa karakter.</p>

<p><img alt="" src="images/2_Tokenization_Complexity.jpg" style="height:500px; width:686px" /><br />
[<a href="http://aclweb.org/anthology/Y/Y11/Y11-1038.pdf" target="_blank"><strong>Image Source</strong></a>]</p>


In [ ]:
# Contoh Tokenizer untuk twitter
Tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
tweet = "@Kirana_Sutanto I am so happpppppppy"
print(Tokenizer.tokenize(tweet))

In [ ]:
# Walaupun fungsi tokenisasi di atas sebenarnya adalah perpaduan beberapa operasi dasar, misal:

tweet = "@Kirana_Sutanto I am so happpppppppppppppy you"
print(''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet)))
# Hikmahnya: Sebagai DS/NLP "terkadang" kita perlu membuat tokenizer custom, 
# terutama untuk masalah yang spesifik. Misal bioinformatics, cryptography, etc.

## Tokenisasi (NLP) Bahasa Indonesia:

<p>NLTK belum support Bahasa Indonesia, bahkan module NLP Python yang support bahasa Indonesia secara umum masih sangat langka. Beberapa <u><strong>resources </strong></u>yang dapat digunakan:</p>

<ol>
	<li><strong><a href="https://github.com/kirralabs/indonesian-NLP-resources" target="_blank">KirraLabs</a></strong>: Mix of NLP-TextMining resources</li>
	<li><strong><a href="https://pypi.python.org/pypi/Sastrawi/1.0.1" target="_blank">Sastrawi 1.0.1</a>:</strong>&nbsp;untuk &quot;stemming&quot; &amp;&nbsp;<strong><a href="https://devtrik.com/python/stopword-removal-bahasa-indonesia-python-sastrawi/" target="_blank">stopwords&nbsp;</a></strong>bahasa Indonesia.</li>
	<li><strong><a href="http://stop-words-list-bahasa-indonesia.blogspot.co.id/2012/09/daftar-kata-dasar-bahasa-indonesia.html" target="_blank">Daftar Kata Dasar Indonesia</a></strong>:&nbsp;Bisa di load sebagai dictionary di Python</li>
	<li><strong><a href="https://id.wiktionary.org/wiki/Wiktionary:ProyekWiki_bahasa_Indonesia/Daftar_kata" target="_blank">Wiktionary</a></strong>: ProyekWiki bahasa Indonesia [termasuk Lexicon]</li>
	<li><a href="http://wn-msa.sourceforge.net/" target="_blank"><strong>WordNet Bahasa Indonesia</strong></a>: Bisa di load&nbsp;sebagai dictionary (atau NLTK<em>*</em>) di Python.</li>
	<li><strong><a href="http://kakakpintar.com/daftar-kata-baku-dan-tidak-baku-a-z-dalam-bahasa-indonesia/" target="_blank">Daftar Kata Baku-Tidak Baku</a></strong>: Bisa di load sebagai dictionary di Python.</li>
	<li><strong><a href="https://spacy.io/" target="_blank">Spacy</a></strong>: Cepat/efisien, MIT License, tapi language model Indonesia masih terbatas.</li>
	<li><a href="http://ufal.mff.cuni.cz/udpipe" target="_blank"><strong>UdPipe</strong></a>: Online request &amp; restricted license (support berbagai bahasa -&nbsp;pemrograman).</li>
</ol>

In [ ]:
# Contoh Tokenisasi dalam bahasa Indonesia dengan Spacy
teks = 'Sore itu, Hamzah melihat kupu-kupu di taman. Ibu membeli oleh-oleh di pasar'
tokenS_id = nlp_id(teks)
print([token.text for token in tokenS_id])

In [ ]:
# Jika dipaksa dengan bahasa inggris
teks = 'Sore itu, Hamzah melihat kupu-kupu di taman. Ibu membeli oleh-oleh di pasar'
tokenS_en = nlp_en(teks)
print([token.text for token in tokenS_en])

<p><u><big><strong>Word Case</strong></big></u><big> (Huruf BESAR/kecil):</big></p>

<ul>
	<li>Untuk menganalisa makna (<em>semantic</em>) dari suatu (frase) kata dan mencari informasi dalam proses textmining, seringnya (*) kita tidak membutuhkan informasi huruf besar/kecil dari kata&nbsp;tersebut.</li>
	<li><em>Text case normaliation</em> dapat dilakukan pada string secara efisien tanpa melalui tokenisasi (mengapa?).</li>
	<li>Namun, bergantung pada analisa teks yang akan digunakan pengguna harus berhati-hati dengan urutan proses (pipelining) dalam preprocessing. Mengapa dan apa contohnya?</li>
</ul>

<p>(*) Coba temukan minimal 2 pengecualian dimana&nbsp; huruf kapital/kecil (case) mempengaruhi makna/pemrosesan teks.</p>

In [ ]:
# Ignore case (huruf besar/kecil)
T = "Hi there!, I am a student. Nice to meet you :)"
print(T.lower())
print(T.upper())
# Perintah ini sangat efisien karena hanya merubah satu bit di setiap (awal) bytes dari setiap karakter
# Sehingga tetap efisien jika ingin dilakukan sebelum tokenisasi

## Morphological-Linguistic Normalization: Stemming &amp; Lemmatization
(Canonical Representation)

## <font color="blue">Stemming dan Lemma</font>

<ol>
	<li>
	<p><strong>Stemmer</strong>&nbsp;akan menghasilkan sebuah bentuk kata yang disepakati oleh suatu sistem tanpa mengindahkan konteks kalimat. Syaratnya beberapa kata dengan makna serupa hanya perlu dipetakan secara konsisten ke sebuah kata baku.&nbsp;Banyak digunakan di IR &amp;&nbsp;komputasinya relatif sedikit. Biasanya dilakukan dengan menghilangkan imbuhan (suffix/prefix).</p>
	</li>
	<li>
	<p><strong>lemmatisation</strong> akan menghasilkan kata baku (dictionary word) dan bergantung konteks.</p>
	</li>
	<li>
	<p>Lemma &amp; stemming bisa jadi sama-sama menghasilkan suatu akar kata (root word). Misal : <em>Melompat </em>==&gt; <em>lompat</em></p>
	</li>
</ol>

<p><strong>Mengapa melakukan Stemming &amp; Lemmatisasi</strong>?</p>

<ol>
	<li>Sering digunakan di IR (Information Retrieval) agar ketika seseorang mencari kata tertentu, maka seluruh kata yang terkait juga diikutsertakan.<br />
	Misal:&nbsp;<em>organize</em>,&nbsp;<em>organizes</em>, and&nbsp;<em>organizing&nbsp;</em>&nbsp;dan&nbsp;<em>democracy</em>,&nbsp;<em>democratic</em>, and&nbsp;<em>democratization</em>.</li>
	<li>Di Text Mining Stemming dan Lemmatisasi akan mengurangi dimensi (mengurangi variasi morphologi), yang terkadang akan meningkatkan akurasi.</li>
	<li>Tapi di IR efeknya malah berkebalikan: <strong><font color="blue">meningkatkan recall, tapi menurunkan akurasi&nbsp;</font></strong>[<a href="https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html" target="_blank"><strong>Link</strong></a>]. Contoh: kata&nbsp;<em>operate, operating, operates, operation, operative, operatives, dan operational</em>&nbsp;jika di stem menjadi <em>operate</em>, maka ketika seseorang mencari &quot;<em>operating system</em>&quot;, maka entry seperti&nbsp;<em>operational and research</em> dan&nbsp;<em>operative and dentistry</em>&nbsp;akan muncul sebagai entry dengan relevansi yang cukup tinggi.</li>
</ol>


<p><strong>Stemming tidak perlu &quot;benar&quot;, hanya perlu konsisten. Sehingga memiliki berbagai variansi, (sebagian) contoh di NLTK:</strong></p>

In [ ]:
# TextBlob Stemming & Lemmatizer 

# Stemming
print(Word('running').stem()) # menggunakan NLTK Porter stemmer

# Lemmatizer
print(Word('ran').lemmatize('v'))

# default Noun, plural akan menjadi singular dari akar katanya
# Juga case sensitive

In [ ]:
# Lemmatizer Indonesia dengan module Sastrawi

I = "perayaan itu Berbarengan dengan saat kita bepergian ke Makassar"
print(stemmer.stem(I))

<h3 id="Tips:">Tips:</h3>

<ul>
	<li>Secara umum &#39;biasanya&#39; di Text Mining yang kita butuhkan hanyalah <strong><font color="blue">Lemma</font></strong>.</li>
	<li>&quot;Kecuali&quot; di aplikasi IR, spelling correction, variasi kata, clustering, atau terkadang klasifikasi. Pada aplikasi-aplikasi tersebut stemming terkadang lebih diinginkan.</li>
	<li>Stemming jauh lebih cepat, tapi tidak selalu tersedia di modul NLP.</li>
	<li>Beberapa algoritma tertentu membutuhkan tanda &quot;.&quot; dan &quot;,&quot; : contohnya untuk document summarization di textRank.</li>
	<li>&quot;_&quot; juga biasa digunakan untuk menyatakan frase kata di representasi n-grams (misal &quot;buah_tangan&quot;).</li>
	<li>Stemming juga digunakan pada Word Sense Disambiguation (WSD)</li>
</ul>

<h3 id="Diskusi:">Diskusi:</h3>

<ul>
	<li>Untuk menghemat storage database, apakah sebaiknya kita menyimpan saja hasil preprocessed texts/documents?</li>
</ul>


### Part-of-Speech (pos) di ilmu bahasa (Linguistik)
<p><img alt="" src="images/2_parts-of-speech-chart.jpg" style="height:400px; width:404px" /></p>
<p>[<a href="https://www.paperrater.com/page/parts-of-speech" target="_blank">image source</a>]</p>

### Daftar  pos-tags di Text Mining:
<img alt="" src="images/2_post_tags_NLTK.png" style="height:400px; width:516px" /></h3>

<p>[<a href="http://gitqwerty777.github.io/natural-language-processing/" target="_blank">image source</a>]</p>

In [ ]:
# Pos tags in TextBlob (English)
for word, pos in TextBlob(T).tags:
    print(word, pos, end=', ')
T

In [ ]:
# Pos Tag Bahasa Indonesia lewat NLTK
# https://yudiwbs.wordpress.com/2018/02/20/pos-tagger-bahasa-indonesia-dengan-pytho/

ct = CRFTagger()
try:
    ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')
except:
    ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
    
hasil = ct.tag_sents([['Saya','bekerja','di','Bandung']])
print(hasil)
# Hati-hati dengan struktur data inputnya

In [ ]:
hasil[0][1][0]

## Text Level Normalization: StopWords
<p><u>Di Text Mining</u> kata-kata yang <strong>sering muncul </strong>(dan jarang sekali muncul) memiliki sedikit sekali informasi (signifikansi) terhadap model (machine learning) yang digunakan. Hal ini di karenakan kata-kata tersebut muncul di semua kategori (di permasalahan klasifikasi) atau di semua cluster (di permasalahan pengelompokan/clustering). Kata-kata yang sering muncul ini biasa disebut &quot;StopWords&quot;. Stopwords berbeda-beda bergantung dari Bahasa dan Environment (aplikasi)-nya.<br />
<strong>Contoh</strong>:<br />

<ul>
	<li>Stopwords bahasa Inggris: am, is, are, do, the, of, etc.</li>
	<li>Stopwords bahasa Indonesia: adalah, dengan, yang, di, ke, dsb</li>
	<li>Stopwords twitter: RT, ...<br />
	<img alt="" src="images/2_StopWords.png" style="height:250px; width:419px" /></li>
</ul>


In [ ]:
# Loading StopWords dari file
try:
    f = 'data/stopwords_id.txt' # atau stopwords_en.txt
except:
    f = 'stopwords_id.txt' # atau stopwords_en.txt
df=open(f,"r",encoding="utf-8", errors='replace')
stopWords = df.readlines();df.close()

print(stopWords[:5])

In [ ]:
stopWords = set([t.strip() for t in stopWords])

In [ ]:
T = "Saya sedang belajar NLP dan text mining"
T = T.lower()
Tokens = TextBlob(T).words
T2 = [t for t in Tokens if str(t) not in stopWords] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
print(' '.join(T2))
# Catatan: Selalu lakukan Stopword filtering setelah tokenisasi (dan normalisasi).

<p><img alt="" src="images/2_Tokenization_Stopwords.png" style="height:400px; width:765px" /></p>

<p>[<a href="http://chdoig.github.io/acm-sigkdd-topic-modeling/#/6/2" target="_blank">image source</a>]</p>


## Preprocessing menghilangkan simbol-simbol.


In [ ]:
# cleaning non alpha-numeric

txt = 'Hi! @Mukidi, apa kabar? #sapa_Pagi.'
print(re.sub(r'[^\w]',' ',txt))
# atau jika ingin exclude titik dan koma 
# re.sub(r'[^.,a-zA-Z0-9 \n\.]','',txt)

In [ ]:
# Cleaning hashTags dalam posting media sosial
tweet = 'oh IoT, #AndaiSajaIaTahu #ApaYangAkuRasah... #AlayersTweet'

getHashtags = re.compile(r"#(\w+)")
pisahtags = re.compile(r'[A-Z][^A-Z]*')

print("Tags = {0}".format(re.findall(getHashtags, tweet)))
# temukan hanya tags ... perhatikan IoT bukan Tags walau ada huruf besar & kecil dalam satu kata

In [ ]:
for tags in re.findall(getHashtags, tweet):
    print(re.findall(pisahtags, tags))

## Encoding-Decoding:

<ul>
	<li>Hal berikutnya yang perlu diperhatikan dalam memproses data teks adalah encoding-decoding.</li>
	<li>Contoh Encoding: ASCII, utf, latin, dsb.</li>
	<li>saya membahas lebih jauh tetang encoding disini:&nbsp;<br />
	<a href="https://tau-data.id/memahami-string-python/" target="_blank">https://tau-data.id/memahami-string-python/</a></li>
	<li>Berikut adalah sebuah contoh sederhana tantangan proses encoding-decoding ketika kita hendak memproses data yang berasal dari internet atau media sosial.</li>
</ul>


In [ ]:
# kita bisa menggunakan modul unidecode untuk mendapatkan representasi ASCII terdekat

T = "ḊḕḀṙ ₲ØĐ, p̾l̾e̾a̾s̾e ḧḕḶṖ ṁḕ"
print(unidecode(T).lower())
# Bahasa Indonesia dan Inggris secara umum mampu direpresentasikan dalam encoding ASCII: 
# https://en.wikipedia.org/wiki/ASCII

In [ ]:
# Kita juga bisa membersihkan posting media sosial/website dengan entitas html menggunakan fungsi "unescape" di modul "html"
print(unescape('Satu &lt; Tiga&nbsp;&amp; &#169; adalah simbol Copyright'))

In [ ]:
T = {'gan':'juragan', 'luv':'love'} # Dictionary
K = 'Gan, kayaknya saya luv banget nih tasnya'
tK = TextBlob(K.lower()).words
tK

In [ ]:
# utk setiap token di tK kalau ada kata-kata yg termasuk di dalam "keys" dari dictionarynya maka ubah menjadi bentuk umumnya.
Kb = []  # kalimat bersih
for token in tK:
    if token in T.keys():
        Kb.append(T[token])
    else:
        Kb.append(token)
' '.join(Kb)

## Loading Dataset dari Modul
contoh dataset dokumen yang cukup tenar: &quot;20 NewsGroup&quot;

<img alt="" src="images/6_20News.jpg" style="height: 300px ; width: 533px" />
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups" target="_blank">http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups</a></p>

<p><strong>Categories </strong>=&nbsp;</p>
<pre>
[&#39;alt.atheism&#39;, &#39;comp.graphics&#39;, &#39;comp.os.ms-windows.misc&#39;, &#39;comp.sys.ibm.pc.hardware&#39;, &#39;comp.sys.mac.hardware&#39;,
 &#39;comp.windows.x&#39;, &#39;misc.forsale&#39;, &#39;rec.autos&#39;, &#39;rec.motorcycles&#39;, &#39;rec.sport.baseball&#39;, &#39;rec.sport.hockey&#39;,
 &#39;sci.crypt&#39;, &#39;sci.electronics&#39;, &#39;sci.med&#39;, &#39;sci.space&#39;, &#39;soc.religion.christian&#39;, &#39;talk.politics.guns&#39;,
 &#39;talk.politics.mideast&#39;, &#39;talk.politics.misc&#39;, &#39;talk.religion.misc&#39;]</pre>


In [ ]:
# Warning: Butuh koneksi internet

categories = ['sci.med', 'talk.politics.misc',  'rec.autos']
data = fetch_20newsgroups(subset='train', categories=categories,remove=('headers', 'footers', 'quotes'))

In [ ]:
# Contoh cara mengakses datanya
print(dir(data))
print(data.data[0]) # Dokumen pertama

In [ ]:
# Rubah struktur data diatas ke dalam bentuk struktur data sederhana: "list of documents"
Y = data.target # List: 0 = class 1, 1 = class 2, ... dst
print('Contoh Label 3 dokumen pertama: ',Y[:3])
X = [doc for doc in data.data] # setiap elemen dalam list adalah dokumen

In [ ]:
len(X)

<h1 id="Vector-Space-Model---VSM">Vector Space Model - VSM</h1>
<p><img alt="" src="images/vsm.png" style="width: 300px; height: 213px;" /></p>

<ul>
	<li>Data yang biasanya kita ketahui berbentuk <strong>tabular </strong>(tabel/kolom-baris/matriks/<em>array</em>/larik), data seperti ini disebut data terstruktur (<strong><em>structured data</em></strong>).</li>
	<li>Data terstruktur dapat disimpan dengan baik di&nbsp;<em>spreadsheet</em>&nbsp;(misal:&nbsp;<em>Excel/CSV</em>) atau basis data (<em>database</em>) relasional dan secara umum dapat digunakan langsung oleh berbagai model/<em>tools</em>&nbsp;statistik/data mining konvensional.</li>
	<li>Sebagian data yang lain memiliki &ldquo;<em>tags</em>&rdquo; yang menjelaskan elemen semantik yang berbeda di dalamnya dan cenderung tidak memiliki skema (struktur) yang statis.</li>
	<li>Data seperti ini disebut data<em>&nbsp;<strong>semi-structured</strong></em>, contohnya data dalam bentuk &nbsp;<strong><a href="http://www.w3.org/XML/" target="_blank">XML</a></strong>.</li>
	<li>Apa bedanya? Apa maksudnya tidak memiliki skema yang statis? Penjelasan mudahnya bayangkan sebuah data terstruktur (tabular), namun dalam setiap baris (<em>record/instance</em>)-nya tidak memiliki jumlah variabel (peubah) yang sama.</li>
	<li>Tentu saja data seperti ini tidak sesuai jika disimpan dan diolah dengan&nbsp;<em>tools/software</em>&nbsp;yang mengasumsikan struktur yang statis pada setiap barisnya (misal: Excel dan SPSS).</li>
</ul>

<p><img alt="" src="images/3_tipeData.png" style="height: 400px ; width: 430px" /></p>


<ul>
	<li>Data multimedia seperti teks, gambar atau video <strong>tidak dapat</strong>&nbsp;<strong>secara langsung</strong>&nbsp;dianalisa dengan model statistik/data mining.</li>
	<li>Sebuah proses awal&nbsp;<em>(pre-process)</em>&nbsp;harus dilakukan terlebih dahulu untuk merubah data-data tidak (semi) terstruktur tersebut menjadi bentuk yang dapat digunakan oleh model statistik/data mining konvensional.</li>
	<li>Terdapat berbagai macam cara mengubah data-data tidak terstruktur tersebut ke dalam bentuk yang lebih sederhana, dan ini adalah suatu bidang ilmu tersendiri yang cukup dalam. Sebagai contoh saja sebuah teks biasanya dirubah dalam bentuk vektor/<em>topics</em>&nbsp;terlebih dahulu sebelum diolah.</li>
	<li>Vektor data teks sendiri bermacam-macam jenisnya: ada yang berdasarkan eksistensi (<strong><em>binary</em></strong>), frekuensi dokumen (<strong>tf</strong>), frekuensi dan invers jumlah dokumennya dalam corpus (<strong><a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf" target="_blank">tf-idf</a></strong>), <strong>tensor</strong>, dan sebagainya.</li>
	<li>&nbsp;Proses perubahan ini sendiri biasanya tidak&nbsp;<em>lossless</em>, artinya terdapat cukup banyak informasi yang hilang. Maksudnya bagaimana? Sebagai contoh ketika teks direpresentasikan dalam vektor (sering disebut sebagai model <strong>bag-of-words</strong>) maka informasi urutan antar kata menghilang.&nbsp;</li>
</ul>

<p><img alt="" src="images/3_structureData.png" style="height:270px; width:578px" /></p>


<p><strong>Contoh bentuk umum representasi dokumen:</strong></p>


<p><img alt="" src="images/3_Bentuk umum representasi dokumen.JPG" style="height: 294px ; width: 620px" /></p>

<p>Pada Model <em>n-grams</em> kolom bisa juga berupa frase.</p>

<h2 id="Document-Term-Matrix-:-Vector-Space-Model---VSM">Document-Term Matrix : Vector Space Model - VSM</h2>

<p><img alt="" src="images/vsm_matrix.png" style="width: 500px; height: 283px;" /></p>


<p><img alt="" src="images/3_tfidf logic.jpg" style="height:359px; width:638px" /></p>

<p><img alt="" src="images/3_variant tfidf.png" style="height:334px; width:955px" /></p>
K = |d|

In [ ]:
# Menggunakan modul SciKit untuk merubah data tidak terstruktur ke VSM
# Scikit implementation http://scikit-learn.org/stable/modules/feature_extraction.html
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [ ]:
# VSM term Frekuensi : "tf"
tf_vectorizer = CountVectorizer(binary = False, lowercase=True, stop_words='english')
tf = tf_vectorizer.fit_transform(X)
print(tf.shape) # Sama

In [ ]:
# VSM term Frekuensi : "tf-idf"
tfidf_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(X)
print(tfidf.shape) # Sama

In [ ]:
tfidf[0].data

In [ ]:
tfidf[0].indices

## Custom tf-idf:
* Menurut http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
* default formula tf-idf yang digunakan sk-learn adalah:
* $tfidf = tf * log(\frac{N}{df+1})$ ==> Smooth IDF
* namun kita merubahnya menjadi:
* $tfidf = tf * log(\frac{N}{df})$ ==> Non Smooth IDF
* $tfidf = tf * log(\frac{N}{df+1})$ ==> Non sublinear_tf, Smooth IDF
* $tfidf = (1+log(tf)) * log(\frac{N}{df})$ ==> sublinear_tf, Non Smooth IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(smooth_idf= False, sublinear_tf=True, lowercase=True, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(X)
print(tfidf.shape) # Sama

In [ ]:
# Frequency Filtering di VSM
tfidf_vectorizer = TfidfVectorizer()
tfidf_1 = tfidf_vectorizer.fit_transform(X)

tfidf_vectorizer = TfidfVectorizer(max_df=0.75, min_df=5)
tfidf_2 = tfidf_vectorizer.fit_transform(X)

print(tfidf_1.shape)
print(tfidf_2.shape)

### Alasan melakukan filtering berdasarkan frekuensi:
* Intuitively filter noise 
* Curse of Dimensionality (akan dibahas kemudian)
* Computational Complexity
* Improving accuracy

In [ ]:
# Variasi pembentukan matriks VSM:
d1 = '@udin76, Minum kopi pagi-pagi sambil makan pisang goreng is the best'
d2 = 'Belajar NLP dan Text Mining ternyata seru banget sadiezz'
d3 =  'Sudah lumayan lama bingits tukang Bakso belum lewat'
d4 = 'Aduh ga banget makan Mie Ayam p4k4i kesyap, please deh'
D = [d1, d2, d3, d4]
# Jika kita menggunakan cara biasa: 
tfidf_vectorizer = TfidfVectorizer()
vsm = tfidf_vectorizer.fit_transform(D)
print(tfidf_vectorizer.vocabulary_)

<p><img alt="" src="images/text-mining-n-gram.jpg" style="width: 800px; height: 400px;" /></p>


In [ ]:
# N-Grams VSM
# Bermanfaat untuk menangkap frase kata, misal: "ga banget", "pisang goreng", dsb
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vsm = tfidf_vectorizer.fit_transform(D)
print(tfidf_vectorizer.vocabulary_)

In [ ]:
# Vocabulary based VSM
# Bermanfaat untuk menghasilkan hasil analisa yang "bersih"
# variasi 2
d1 = '@udin76, Minum kopi pagi-pagi sambil makan pisang goreng is the best'
d2 = 'Belajar NLP dan Text Mining ternyata seru banget sadiezz'
d3 =  'Sudah lumayan lama bingits tukang Bakso belum lewat seru'
d4 = 'Aduh ga banget makan Mie Ayam p4k4i kesyap, please deh'
D = [d1,d2,d3,d4]
Vocab = {'seru banget':0, 'seru':1, 'the best':2, 'lama':3, 'text mining':4, 'nlp':5, 'ayam':6}
tf_vectorizer = CountVectorizer(binary = False, vocabulary=Vocab,ngram_range=(1, 2))
tf = tf_vectorizer.fit_transform(D)
print(tf.toarray())
tf_vectorizer.vocabulary_

# <center><font color="blue"> End of Module

<hr />